In [1]:
from gurobipy import *
import numpy as np

In [2]:
I = ['item1', 'item2', 'item3','item4']        # Items
J = ['sup1', 'sup2', 'sup3']           # Suppliers
K = ['odw1', 'odw2','odw3']                   # On-demand warehouses
R = ['ret1','ret2','ret3']                           # Retailer warehouse
E = ['em1','em2','em3']                            # Emergency warehouse
T = [0, 1, 2]                          # Time periods
M = [1, 2]                             # Commitment periods
S = ['sc1', 'sc2', 'sc3']              # Scenarios

alpha = 100                       # base rental cost per period
gamma = 0.95                      # discount factor for longer commitments
F = {'sup1': 200, 'sup2': 300, 'sup3':300}     # supplier fixed cost
C_k = {'odw1': 1000, 'odw2': 900,'odw3':1200} # odw capacities
C_r = {'ret1': 800,'ret2':850,'ret3':750}                # retailer capacities
C_e = {'em1': 800,'em2':700,'em3':760}                 # emergency capacities

h_r = {'item1': 2, 'item2': 3, 'item3': 4,'item4':5}     # holding cost at retailer
h_k = {'item1': 5, 'item2': 6, 'item3': 7,'item4':8}     # holding cost at odw
h_e = {'item1': 10, 'item2': 11, 'item3': 12,'item4':13}   # holding cost at emergency

b = {'item1': 1, 'item2': 2, 'item3': 3,'item4':4,}      # delivery cost
beta = {'item1': 100, 'item2': 110, 'item3': 120,'item4':130}# lost sales penalty

c_rj = 20                          # cost from supplier to retailer
c_kj = 25                          # cost from supplier to odw
c_ej = 40                          # cost from supplier to emergency

L_s = 1                            # supplier lead time
L_d = 1                            # delivery lead time

# Random demand and supply (1 scenario)
D = {(i, t, s): np.random.randint(20, 60) for i in I for t in T for s in S}
S_supply = {(i, j, t, s): np.random.randint(50, 120) for i in I for j in J for t in T for s in S}
P_s = {
    'sc1': 0.5,
    'sc2': 0.4,
    'sc3': 0.2,
}


In [3]:
m = Model("ODW model")

Restricted license - for non-production use only - expires 2026-11-23


In [4]:
# First-stage variables
y = m.addVars(J, vtype=GRB.BINARY, name="y")
g = m.addVars(K, M, T, vtype=GRB.BINARY, name="g")
r = m.addVars(K, M, T, vtype=GRB.BINARY, name="r")

In [5]:
# Second-stage variables
x_r = m.addVars(I, J, R, T, S, lb=0, name="x_r")
x_k = m.addVars(I, J, K, T, S, lb=0, name="x_k")
x_e = m.addVars(I, J, E, T, S, lb=0, name="x_e")

v_r = m.addVars(I, R, T, S, lb=0, name="v_r")
v_k = m.addVars(I, K, T, S, lb=0, name="v_k")
v_e = m.addVars(I, E, T, S, lb=0, name="v_e")

u_r = m.addVars(I, R, T, S, lb=0, name="u_r")
u_k = m.addVars(I, K, T, S, lb=0, name="u_k")
u_e = m.addVars(I, E, T, S, lb=0, name="u_e")

z = m.addVars(I, T, S, lb=0, name="z")

In [6]:
commitment_cost = quicksum(m_ * alpha * gamma**m_ * g[k, m_, t] 
                         for k in K for m_ in M for t in T)

supplier_investment_cost = quicksum(F[j] * y[j] for j in J)

delivery_cost = quicksum(P_s[s] * b[i] * ( quicksum(u_r[i, r, t, s] for r in R) + quicksum(u_k[i, k, t, s] for k in K) + quicksum(u_e[i, e, t, s] for e in E)) for i in I for t in T for s in S)

stockout_cost = quicksum(P_s[s] * beta[i] * z[i, t, s] 
                       for i in I for t in T for s in S)

transport_cost = quicksum(P_s[s] * (
    quicksum(c_rj * x_r[i, j, r, t, s] for r in R) +
    quicksum(c_kj * x_k[i, j, k, t, s] for k in K) +
    quicksum(c_ej * x_e[i, j, e, t, s] for e in E))
for i in I for j in J for t in T for s in S)

inventory_holding_cost = quicksum(P_s[s] * (
    quicksum(h_r[i] * v_r[i, r, t, s] for r in R) +
    quicksum(h_k[i] * v_k[i, k, t, s] for k in K) +
    quicksum(h_e[i] * v_e[i, e, t, s] for e in E)) for i in I for t in T for s in S)


In [7]:
total_cost = (commitment_cost + supplier_investment_cost + delivery_cost +
             stockout_cost + transport_cost + inventory_holding_cost)

m.setObjective(total_cost, GRB.MINIMIZE)



In [8]:
#commitment cost

for k in K:
    for t in T:
        for m_ in M:
            start = max(0, t - m_ + 1)
            end = t
            tt_range = range(start, end + 1)
            m.addConstr(quicksum(g[k, m_, tt] for tt in tt_range) == r[k, m_, t],
                        f"Commitment_Active_{k}_{m_}_{t}")
        
        m.addConstr(quicksum(r[k, m_, t] for m_ in M) <= 1, 
                   f"Active_Commitment_Limit_{k}_{t}")
        
        m.addConstr(quicksum(g[k, m_, t] for m_ in M) <= 1, 
                   f"New_Commitment_Limit_{k}_{t}")

In [9]:
for s in S:
    # Supplier capacity constraints
    for i in I:
        for j in J:
            for t in T:
                m.addConstr(quicksum(x_r[i, j, r, t, s] for r in R) +
                            quicksum(x_k[i, j, k, t, s] for k in K) +
                            quicksum(x_e[i, j, e, t, s] for e in E) <= 
                            S_supply[i, j, t, s] * y[j],
                           f"Supply_Capacity_{i}_{j}_{t}_{s}")
    
    # Warehouse inventory capacity constraints
    for t in T:
        # Retailer warehouse
        for r_val in R:
            m.addConstr(quicksum(v_r[i, r_val, t, s] for i in I) <= C_r[r_val],
                       f"Retailer_Capacity_{r_val}_{t}_{s}")
        
        # Emergency warehouse
        for e_val in E:
            m.addConstr(quicksum(v_e[i, e_val, t, s] for i in I) <= C_e[e_val],
                       f"Emergency_Capacity_{e_val}_{t}_{s}")
        
        # ODW capacity (depends on commitment)
        for k_val in K:
            m.addConstr(quicksum(v_k[i, k_val, t, s] for i in I) <= 
                       quicksum(r[k_val, m_, t] * C_k[k_val] for m_ in M),
                       f"ODW_Capacity_{k_val}_{t}_{s}")
    
    # Flow capacity constraints
    for t in T:
        # Retailer warehouse
        for r_val in R:
            m.addConstr(quicksum(x_r[i, j, r_val, t, s] for i in I for j in J) <= C_r[r_val],
                       f"Flow_Capacity_Retailer_{r_val}_{t}_{s}")
        
        # Emergency warehouse
        for e_val in E:
            m.addConstr(quicksum(x_e[i, j, e_val, t, s] for i in I for j in J) <= C_e[e_val],
                       f"Flow_Capacity_Emergency_{e_val}_{t}_{s}")
        
        # ODW warehouse
        for k_val in K:
            m.addConstr(quicksum(x_k[i, j, k_val, t, s] for i in I for j in J) <= 
                       C_k[k_val] * quicksum(r[k_val, m_, t] for m_ in M),
                       f"Flow_Capacity_ODW_{k_val}_{t}_{s}")
    
    # Inventory balance constraints
    for i in I:
        # Retailer warehouses
        for r_val in R:
            for t in T:
                # Initial inventory (t=0)
                if t == 0:
                    prev_inv = 0
                else:
                    prev_inv = v_r[i, r_val, t-1, s]
                
                # New shipments arriving (considering supplier lead time)
                if t >= L_s:
                    new_shipments = quicksum(x_r[i, j, r_val, t - L_s, s] for j in J)
                else:
                    new_shipments = 0
                
                m.addConstr(v_r[i, r_val, t, s] == prev_inv + new_shipments - u_r[i, r_val, t, s],
                           f"Inv_Balance_Retailer_{i}_{r_val}_{t}_{s}")
        
        # On-demand warehouses
        for k_val in K:
            for t in T:
                if t == 0:
                    prev_inv = 0
                else:
                    prev_inv = v_k[i, k_val, t-1, s]
                
                if t >= L_s:
                    new_shipments = quicksum(x_k[i, j, k_val, t - L_s, s] for j in J)
                else:
                    new_shipments = 0
                
                m.addConstr(v_k[i, k_val, t, s] == prev_inv + new_shipments - u_k[i, k_val, t, s],
                           f"Inv_Balance_ODW_{i}_{k_val}_{t}_{s}")
        
        # Emergency warehouses
        for e_val in E:
            for t in T:
                if t == 0:
                    prev_inv = 0
                else:
                    prev_inv = v_e[i, e_val, t-1, s]
                
                if t >= L_s:
                    new_shipments = quicksum(x_e[i, j, e_val, t - L_s, s] for j in J)
                else:
                    new_shipments = 0
                
                m.addConstr(v_e[i, e_val, t, s] == prev_inv + new_shipments - u_e[i, e_val, t, s],
                           f"Inv_Balance_Emergency_{i}_{e_val}_{t}_{s}")
    
    # Demand satisfaction with delivery lead time
    for i in I:
        for t in T:
            # Deliveries sent at time t - L_d arrive at time t
            if t >= L_d:
                total_delivery = (quicksum(u_r[i, r, t - L_d, s] for r in R) +
                                quicksum(u_k[i, k, t - L_d, s] for k in K) +
                                quicksum(u_e[i, e, t - L_d, s] for e in E))
            else:
                total_delivery = 0
            
            m.addConstr(total_delivery + z[i, t, s] >= D[i, t, s],
                       f"Demand_Satisfaction_{i}_{t}_{s}")

In [14]:
m.write("ODW.lp")
m.write("ODW.mps")

In [11]:
print("Total variables:", m.numVars)
print("Total constraints:", m.numConstrs)

Total variables: 1695
Total constraints: 666


In [12]:
m.optimize()

Gurobi Optimizer version 12.0.2 build v12.0.2rc0 (win64 - Windows 11.0 (26100.2))

CPU model: 13th Gen Intel(R) Core(TM) i7-13700HX, instruction set [SSE2|AVX|AVX2]
Thread count: 16 physical cores, 24 logical processors, using up to 24 threads

Optimize a model with 666 rows, 1695 columns and 4326 nonzeros
Model fingerprint: 0x8e3b18b6
Variable types: 1656 continuous, 39 integer (39 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+03]
  Objective range  [2e-01, 3e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 9e+02]
Found heuristic solution: objective 63369.000000
Presolve removed 618 rows and 1644 columns
Presolve time: 0.00s
Presolved: 48 rows, 51 columns, 120 nonzeros
Variable types: 48 continuous, 3 integer (3 binary)

Root relaxation: objective 4.593290e+04, 25 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Ti

In [13]:
if m.status == GRB.OPTIMAL:
    print("Optimal objective value:", m.objVal)
    print(f"Total Cost: {m.objVal:.2f}\n")

    print("Cost Breakdown ")
    print(f"Commitment Cost: {commitment_cost.getValue():.2f}")
    print(f"Supplier Investment Cost: {supplier_investment_cost.getValue():.2f}")
    print(f"Delivery Cost: {delivery_cost.getValue():.2f}")
    print(f"Stockout Cost: {stockout_cost.getValue():.2f}")
    print(f"Transport Cost: {transport_cost.getValue():.2f}")
    print(f"Inventory Holding Cost: {inventory_holding_cost.getValue():.2f}\n")
    
    print("\nSupplier selection (y):")
    for j in J:
        if y[j].X > 0.5:
            print(f"  Supplier {j} selected (y={y[j].X})")
    
    print("\nCommitment decisions (g):")
    for k in K:
        for m_ in M:
            for t in T:
                if g[k, m_, t].X > 0.5:
                    print(f"  ODW {k} commitment start at time {t} for period {m_} (g={g[k,m_,t].X})")
    
    print("\nShipments to Retailer (x_r):")
    for i in I:
        for j in J:
            for r_val in R:
                for t in T:
                    for s in S:
                        if x_r[i, j, r_val, t, s].X > 1e-6:
                            print(f"  Item {i}, Supplier {j} -> Retailer {r_val}, Time {t}, Scenario {s}: {x_r[i,j,r_val,t,s].X}")
    
    print("\nLost sales (z):")
    for i in I:
        for t in T:
            for s in S:
                if z[i, t, s].X > 1e-6:
                    print(f"  Item {i}, Time {t}, Scenario {s}, unmet sales: {z[i,t,s].X}")
else:
    print("No optimal solution found.")


Optimal objective value: 45932.9
Total Cost: 45932.90

Cost Breakdown 
Commitment Cost: 0.00
Supplier Investment Cost: 200.00
Delivery Cost: 463.90
Stockout Cost: 41423.00
Transport Cost: 3846.00
Inventory Holding Cost: 0.00


Supplier selection (y):
  Supplier sup1 selected (y=1.0)

Commitment decisions (g):

Shipments to Retailer (x_r):
  Item item1, Supplier sup1 -> Retailer ret2, Time 0, Scenario sc1: 55.0
  Item item1, Supplier sup1 -> Retailer ret3, Time 0, Scenario sc2: 42.0
  Item item1, Supplier sup1 -> Retailer ret3, Time 0, Scenario sc3: 22.0
  Item item2, Supplier sup1 -> Retailer ret1, Time 0, Scenario sc2: 56.0
  Item item2, Supplier sup1 -> Retailer ret2, Time 0, Scenario sc1: 55.0
  Item item2, Supplier sup1 -> Retailer ret3, Time 0, Scenario sc3: 24.0
  Item item3, Supplier sup1 -> Retailer ret1, Time 0, Scenario sc1: 56.0
  Item item3, Supplier sup1 -> Retailer ret2, Time 0, Scenario sc3: 59.0
  Item item3, Supplier sup1 -> Retailer ret3, Time 0, Scenario sc2: 25.0
  